# PTT爬蟲

In [53]:
!mkdir -p /tmp/ptt_data/list_page

In [54]:
!mkdir -p /tmp/ptt_data/detail_page

In [55]:
import requests

def get_total_page():
    resp = requests.get("https://www.ptt.cc/bbs/Gossiping/index.html", headers={'cookie': "over18=1"})
    text = resp.text
    return int(text.split('a class="btn wide" href="/bbs/Gossiping/index')[2].split('.html">&lsaquo; 上頁</a>')[0]) + 1

def get_page(url):
    """
    args: 
          url<str>: url to crawl
    return:
          res<str>: text of the webpage
    """
    resp = requests.get(url, headers={'cookie': "over18=1"})
    return resp.text

def save2file(filename, text):
    """
    args:
          filename<str>: filename of the file
          text<str>: text content to be saved
    return:
          
    """
    with open(filename, 'w') as f:
        f.write(text)

In [56]:
url_template = "https://www.ptt.cc/bbs/Gossiping/index%s.html"
total_page = get_total_page()
pages_to_crawl = 2
for page in range(total_page, total_page - pages_to_crawl,-1):
    url = url_template%page
    print("[INFO]crawling %s"%url)
    text = get_page(url)
    save2file('/tmp/ptt_data/list_page/' + "%s.html"%page, text)

[INFO]crawling https://www.ptt.cc/bbs/Gossiping/index39057.html
[INFO]crawling https://www.ptt.cc/bbs/Gossiping/index39056.html


In [57]:
# glob is like ls for Python
# finds file paths
import glob

glob.glob('/tmp/ptt_data/list_page/*')

res = []
for filename in glob.glob('/tmp/ptt_data/list_page/*'):
    with open(filename) as f:
        html = f.read()
        detail_urls = re.findall('''/bbs/Gossiping/M.+html''', html)
        res.extend(detail_urls)

res

['/bbs/Gossiping/M.1522112372.A.E71.html',
 '/bbs/Gossiping/M.1522112378.A.AE4.html',
 '/bbs/Gossiping/M.1522112405.A.B2C.html',
 '/bbs/Gossiping/M.1522112478.A.5CD.html',
 '/bbs/Gossiping/M.1522112485.A.921.html',
 '/bbs/Gossiping/M.1522112503.A.1BD.html',
 '/bbs/Gossiping/M.1522112553.A.D9A.html',
 '/bbs/Gossiping/M.1522112604.A.138.html',
 '/bbs/Gossiping/M.1522112646.A.450.html',
 '/bbs/Gossiping/M.1522112685.A.A97.html',
 '/bbs/Gossiping/M.1522112686.A.54F.html',
 '/bbs/Gossiping/M.1522112726.A.8E3.html',
 '/bbs/Gossiping/M.1522112834.A.FA2.html',
 '/bbs/Gossiping/M.1522112856.A.FB2.html',
 '/bbs/Gossiping/M.1522112875.A.207.html',
 '/bbs/Gossiping/M.1522112889.A.B16.html',
 '/bbs/Gossiping/M.1522112890.A.BFE.html',
 '/bbs/Gossiping/M.1522112913.A.FD0.html',
 '/bbs/Gossiping/M.1522112922.A.949.html',
 '/bbs/Gossiping/M.1522108695.A.B98.html',
 '/bbs/Gossiping/M.1522108824.A.79D.html',
 '/bbs/Gossiping/M.1522108900.A.E6B.html',
 '/bbs/Gossiping/M.1522108938.A.21D.html',
 '/bbs/Goss

In [59]:
for link in res:
    filename = link.split('/')[-1].replace('.html','').replace('.', '_') + '.html'
    url = 'https://www.ptt.cc' + link
    print(url)
    
    # using the fucntions defined above ^^^^^^^^^^^^^
    text = get_page(url)
    save2file('/tmp/ptt_data/detail_page/' + filename, text)

In [7]:
# GET Request
import requests

url = "http://ianchenhq.com"

resp = requests.get(url)

resp

<Response [200]>

In [8]:
resp.status_code

200

In [9]:
resp.text

'\n<!doctype html>\n<html lang="zh-Hant">\n  <head>\n    <!-- Required meta tags -->\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n\n    <!-- Bootstrap CSS -->\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">\n    <!-- Custom styles for this template -->\n    <link href="/static/carousel.css" rel="stylesheet">\n    \n    <title>Ian Chen - 大數據課程網站 - Python</title>\n  </head>\n  <body>\n    <header>\n      <nav class="navbar navbar-expand-md navbar-dark bg-dark">\n        <a class="navbar-brand" href="#">Ian Chen</a>\n        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarCollapse" aria-controls="navbarCollapse" aria-expanded="false" aria-label="Toggle navigation">\n          <span class="navbar-toggler-icon"></

# HTML Parsing

In [19]:
import glob

detail_page = glob.glob('/tmp/ptt_data/detail_page/*')[0]

print(detail_page)

with open(detail_page) as f:
    html = f.read()
    
html

/tmp/ptt_data/detail_page/M_1522114145_A_ADB.html


'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>[新聞] 綠執政親日 農委會成重災區 - 看板 Gossiping - 批踢踢實業坊</title>\n<meta name="robots" content="all">\n<meta name="keywords" content="Ptt BBS 批踢踢">\n<meta name="description" content="1.媒體來源:\n※ 例如蘋果日報、奇摩新聞\n風傳媒\n2.完整新聞標題:\n※ 標題沒有寫出來 ---&gt; 依照板規刪除文章\n">\n<meta property="og:site_name" content="Ptt 批踢踢實業坊">\n<meta property="og:title" content="[新聞] 綠執政親日 農委會成重災區">\n<meta property="og:description" content="1.媒體來源:\n※ 例如蘋果日報、奇摩新聞\n風傳媒\n2.完整新聞標題:\n※ 標題沒有寫出來 ---&gt; 依照板規刪除文章\n">\n<link rel="canonical" href="https://www.ptt.cc/bbs/Gossiping/M.1522114145.A.ADB.html">\n\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-common.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-base.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-custom.css">\n<link rel="style

In [10]:
!pip install beautifulsoup4

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
from bs4 import BeautifulSoup


soup = BeautifulSoup(html, 'lxml') # html5lib

In [22]:
soup.select_one('span.article-meta-value')

<span class="article-meta-value">oridio (新手上路)</span>

In [30]:
metas = soup.select('span.article-meta-value')

metas # [<Tag>, <Tag>, <Tag>, <Tag>]

[<span class="article-meta-value">oridio (新手上路)</span>,
 <span class="article-meta-value">Gossiping</span>,
 <span class="article-meta-value">[新聞] 綠執政親日 農委會成重災區</span>,
 <span class="article-meta-value">Tue Mar 27 09:29:02 2018</span>]

In [31]:
"""
{
"title": "[新聞] 綠執政親日 農委會成重災區",
"author": "oridio (新手上路)",
"board": "Gossiping"
}

"""
article_dict = {}

# metas[0] -> <Tag>
# metas[0].text => Tag.text -> <str>
article_dict['author'] = metas[0].text
article_dict['board']  = metas[1].text
article_dict['title']  = metas[2].text
article_dict['dt']     = metas[3].text

article_dict

{'author': 'oridio (新手上路)',
 'board': 'Gossiping',
 'dt': 'Tue Mar 27 09:29:02 2018',
 'title': '[新聞] 綠執政親日 農委會成重災區'}

In [35]:
import re

re.findall(', 來自: 39.9.68.12\n', html)

[', 來自: 39.9.68.12\n']

In [36]:
re.findall(', 來自: (39.9.68.12)\n', html)

['39.9.68.12']

In [37]:
re.findall(', 來自: (\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})\n', html)

['39.9.68.12']

In [39]:
article_dict['ip'] = re.findall(', 來自: (\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})\n', html)[0]
article_dict

{'author': 'oridio (新手上路)',
 'board': 'Gossiping',
 'dt': 'Tue Mar 27 09:29:02 2018',
 'ip': '39.9.68.12',
 'title': '[新聞] 綠執政親日 農委會成重災區'}

In [47]:
main_content = soup.select_one('#main-content')

for tag in main_content.select('.article-metaline'):
    tag.extract()
    
for tag in main_content.select('.article-metaline-right'):
    tag.extract()

In [48]:
main_content

<div class="bbs-screen bbs-content" id="main-content">

1.媒體來源:
<span class="f2">※ 例如蘋果日報、奇摩新聞</span>
風傳媒

2.完整新聞標題:
<span class="f2">※ 標題沒有寫出來 ---&gt; 依照板規刪除文章</span>
綠營執政農業政策親日　農委會成文官重災區 12個局處首長11個異動

3.完整新聞內文:
<span class="f2">※ 社論特稿都不能貼! 違者刪除(政治類水桶3個月)，貼廣告也會被刪除喔!</span>
民進黨重新執政後，台日關係明顯升溫，綠營政治人物日本考察絡繹於途，台灣蜜棗、高麗菜最近成功銷日，日本農產品，除了和牛開放進口外，接著核災區食品未來配合台灣加入CPTPP（跨太平洋夥伴全面進展協定），很可能配合開放，台日雙方在農業交流議題，檯面上給足了面子，檯面下卻苦了底下的技術官僚，從綠營執政迄今，農委會12個局處一級主管，已經有11個異動，成為政黨輪替的重災區。

民進黨政府與日本安倍政府意識形態接近，雙方關係友好，其實並不算新聞，台灣在2011年大地震對日本的慷慨捐輸，更讓日本從官方到民間都感念在心，為了鞏固雙邊策略夥伴關係，台日兩國最近在農業政策上，給予對方明顯讓步，雙方緊密關係不言可諭。

今年三月，高雄市長陳菊與農委會主委林聰賢，連袂出席日本東京食品展，為台灣南部蜜棗首度成功銷日站台，林聰賢的臉書上寫到，「這顆大到讓日本人以為是青蘋果的蜜棗，是我們農委會高雄農改場所改良的品種，從現在起，在日本海老名高速公路休息站也品嘗的到。為了這一天，我們台灣和日本談了7年半，最主要關鍵是日本要求的檢疫條件。」

說服日本開放台灣鮮果進口 農委會檢疫處承擔成敗責任

林聰賢所提及的「檢疫條件」，主要是因為台灣是東方果實蠅疫區，日方對台灣生鮮果實的檢疫要求嚴謹，日方先前係以台灣在國際文獻中列為「番石榴果實蠅」疫區為由禁止台灣水果進口，為了證明台灣蜜棗並無病蟲害問題，農委會方面花了很多功夫說服日方，終於在去年底獲得日方同意進口。

由於農產品解禁，必須由二國農政單位共同把關，不過為了讓綠營高層能夠趕上3月初東京食品展，農委會防檢局承受頗大壓力，尷尬的是，先前通過台灣檢疫單位檢驗的3批蜜棗，出口日本後，卻有一批一度檢疫有問題，讓農委會大為

In [49]:
for tag in main_content.select('div.push'):
    tag.extract()

In [50]:
main_content

<div class="bbs-screen bbs-content" id="main-content">

1.媒體來源:
<span class="f2">※ 例如蘋果日報、奇摩新聞</span>
風傳媒

2.完整新聞標題:
<span class="f2">※ 標題沒有寫出來 ---&gt; 依照板規刪除文章</span>
綠營執政農業政策親日　農委會成文官重災區 12個局處首長11個異動

3.完整新聞內文:
<span class="f2">※ 社論特稿都不能貼! 違者刪除(政治類水桶3個月)，貼廣告也會被刪除喔!</span>
民進黨重新執政後，台日關係明顯升溫，綠營政治人物日本考察絡繹於途，台灣蜜棗、高麗菜最近成功銷日，日本農產品，除了和牛開放進口外，接著核災區食品未來配合台灣加入CPTPP（跨太平洋夥伴全面進展協定），很可能配合開放，台日雙方在農業交流議題，檯面上給足了面子，檯面下卻苦了底下的技術官僚，從綠營執政迄今，農委會12個局處一級主管，已經有11個異動，成為政黨輪替的重災區。

民進黨政府與日本安倍政府意識形態接近，雙方關係友好，其實並不算新聞，台灣在2011年大地震對日本的慷慨捐輸，更讓日本從官方到民間都感念在心，為了鞏固雙邊策略夥伴關係，台日兩國最近在農業政策上，給予對方明顯讓步，雙方緊密關係不言可諭。

今年三月，高雄市長陳菊與農委會主委林聰賢，連袂出席日本東京食品展，為台灣南部蜜棗首度成功銷日站台，林聰賢的臉書上寫到，「這顆大到讓日本人以為是青蘋果的蜜棗，是我們農委會高雄農改場所改良的品種，從現在起，在日本海老名高速公路休息站也品嘗的到。為了這一天，我們台灣和日本談了7年半，最主要關鍵是日本要求的檢疫條件。」

說服日本開放台灣鮮果進口 農委會檢疫處承擔成敗責任

林聰賢所提及的「檢疫條件」，主要是因為台灣是東方果實蠅疫區，日方對台灣生鮮果實的檢疫要求嚴謹，日方先前係以台灣在國際文獻中列為「番石榴果實蠅」疫區為由禁止台灣水果進口，為了證明台灣蜜棗並無病蟲害問題，農委會方面花了很多功夫說服日方，終於在去年底獲得日方同意進口。

由於農產品解禁，必須由二國農政單位共同把關，不過為了讓綠營高層能夠趕上3月初東京食品展，農委會防檢局承受頗大壓力，尷尬的是，先前通過台灣檢疫單位檢驗的3批蜜棗，出口日本後，卻有一批一度檢疫有問題，讓農委會大為

In [52]:
article_dict['content'] = main_content.text.strip()

article_dict

{'author': 'oridio (新手上路)',
 'board': 'Gossiping',
 'content': '1.媒體來源:\n※ 例如蘋果日報、奇摩新聞\n風傳媒\n\n2.完整新聞標題:\n※ 標題沒有寫出來 ---> 依照板規刪除文章\n綠營執政農業政策親日\u3000農委會成文官重災區 12個局處首長11個異動\n\n3.完整新聞內文:\n※ 社論特稿都不能貼! 違者刪除(政治類水桶3個月)，貼廣告也會被刪除喔!\n民進黨重新執政後，台日關係明顯升溫，綠營政治人物日本考察絡繹於途，台灣蜜棗、高麗菜最近成功銷日，日本農產品，除了和牛開放進口外，接著核災區食品未來配合台灣加入CPTPP（跨太平洋夥伴全面進展協定），很可能配合開放，台日雙方在農業交流議題，檯面上給足了面子，檯面下卻苦了底下的技術官僚，從綠營執政迄今，農委會12個局處一級主管，已經有11個異動，成為政黨輪替的重災區。\n\n民進黨政府與日本安倍政府意識形態接近，雙方關係友好，其實並不算新聞，台灣在2011年大地震對日本的慷慨捐輸，更讓日本從官方到民間都感念在心，為了鞏固雙邊策略夥伴關係，台日兩國最近在農業政策上，給予對方明顯讓步，雙方緊密關係不言可諭。\n\n今年三月，高雄市長陳菊與農委會主委林聰賢，連袂出席日本東京食品展，為台灣南部蜜棗首度成功銷日站台，林聰賢的臉書上寫到，「這顆大到讓日本人以為是青蘋果的蜜棗，是我們農委會高雄農改場所改良的品種，從現在起，在日本海老名高速公路休息站也品嘗的到。為了這一天，我們台灣和日本談了7年半，最主要關鍵是日本要求的檢疫條件。」\n\n說服日本開放台灣鮮果進口 農委會檢疫處承擔成敗責任\n\n林聰賢所提及的「檢疫條件」，主要是因為台灣是東方果實蠅疫區，日方對台灣生鮮果實的檢疫要求嚴謹，日方先前係以台灣在國際文獻中列為「番石榴果實蠅」疫區為由禁止台灣水果進口，為了證明台灣蜜棗並無病蟲害問題，農委會方面花了很多功夫說服日方，終於在去年底獲得日方同意進口。\n\n由於農產品解禁，必須由二國農政單位共同把關，不過為了讓綠營高層能夠趕上3月初東京食品展，農委會防檢局承受頗大壓力，尷尬的是，先前通過台灣檢疫單位檢驗的3批蜜棗，出口日本後，卻有一批一度檢疫有問題，讓農委會大為緊張。\n\n農委會官員表示，日本之所以開放台灣蜜棗銷日，台日人民感情因素扮演頗關鍵